In [ ]:
from matplotlib.colors import SymLogNorm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates.sky_coordinate import SkyCoord
import astropy.units as u
#from astropy.cosmology import Planck15 as cosmo
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725, Ob0 = 0.0486, m_nu = [0, 0, 0.06] * u.eV)
from astropy.wcs import WCS
from minot import model

# Modify plotting parameters
dict_base = {'font.size':        16,
             'legend.fontsize':  16,
             'xtick.labelsize':  16,
             'ytick.labelsize':  16,
             'axes.labelsize':   16,
             'axes.titlesize':   16,
             'figure.titlesize': 16,    
             'figure.figsize':[8.0, 6.0],
             'figure.subplot.right':0.97,
             'figure.subplot.left':0.15,
             'font.family':'serif',
             'figure.facecolor': 'white',
             'legend.frameon': True}
plt.rcParams.update(dict_base)

In [ ]:
clust = model.Cluster(name='A2151CB', 
                      redshift=0.0368, M500=9.08e13*u.Msun, 
                      cosmology=cosmo, silent=False, 
                      output_dir='/Users/vdk/Desktop/A2151CB_TestLessEmax')

In [ ]:
#----- Some parameters that can be set at when defining the cluster
clust.coord = SkyCoord("241.14d  17.72d", frame="icrs", unit="deg")

#----- Define the relevant ICM model (available models are defined in model_modpar.py)

# Density profile of the target thermal gas
clust.density_gas_model = {'name':'beta', 'n_0':8.75e-3*u.cm**-3, 'beta':0.38, 'r_c':19.35*u.kpc}

#UPP Plank P0 total
clust.pressure_gas_model = {'name':'GNFW', 'P_0':0.009*u.keV/u.cm**3, 'c500':1.81, 'a':1.33, 'b':4.13, 'c':0.31}

#UPP Planck P0 From Tiwari
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':0.005*u.keV/u.cm**3, 'c500':1.81, 'a':1.33, 'b':4.13, 'c':0.31}

# Pressure profile of the target thermal gas
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.2e-2*u.keV/u.cm**3, 'c500':3.2, 'a':1.5, 'b':3.1, 'c':0.0}

#Arnaud test
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.2e-2*u.keV/u.cm**3, 'c500':1.1, 'a':1., 'b':5.4, 'c':0.3}

#Gong test
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.99e-2*u.keV/u.cm**3, 'c500':1.16, 'a':2.66, 'b':5.48, 'c':0.3}

#Test from minot model
#clust.set_pressure_gas_gNFW_param(pressure_model = 'P13UPP')

#Planck test
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.57e-2*u.keV/u.cm**3, 'c500':1.17, 'a':1.05, 'b':5.49, 'c':0.3}

#Pressure from Nagai paper
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':3.3e-2*u.keV/u.cm**3, 'c500':1.8, 'a':1.3, 'b':4.3, 'c':0.7}

# Set the cosmic ray density profile to the same shape as the thermal gas density
clust.set_density_crp_isodens_scal_param()
clust.set_density_cre1_isodens_scal_param()

# Cosmic ray proton spectrum
clust.spectrum_crp_model = {'name':'ExponentialCutoffPowerLaw', 'Index':1.5, 'CutoffEnergy':30*u.TeV}

# Cosmic ray proton normalization
clust.X_crp_E = {'X':0.1, 'R_norm':clust.R500} # X is the CRp/thermal energy ratio, normalized within R_norm

clust.X_cre1_E = {'X':0.01, 'R_norm':clust.R500} # X is the CRp/thermal energy ratio, normalized within R_norm



#Set magnetic field profile
clust.set_magfield_isodens_scal_param(10000000*u.uG, scal=2/3)

clust.Epmax = 10e3*u.TeV
clust.Eemax = 10e3*u.TeV

clust.abundance = 0.43

In [ ]:
def plot_CTA_sens():

    # CTA Performance - Northern site
    # for further details, see: www.cta - observatory.org / science / cta - performance / (prod3b - v2)

    bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
    bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
    data = [2.37e-11, 8.35e-12, 2.02e-12, 9.41e-13, 5.37e-13, 3.58e-13, 2.43e-13, 1.76e-13, 1.35e-13, 1.24e-13, 1.1e-13,
            1.21e-13, 1.44e-13, 2.06e-13, 3.2e-13, 5.1e-13, 7.97e-13, 1.2e-12, 1.93e-12, 3.92e-12] # S (erg / cm^2 / s)
    data_GeV = [i * 624.15064799632 for i in data]
    mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]


    plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5)
    plt.show()

plot_CTA_sens()

In [ ]:
clust.print_param()

In [ ]:
plt.figure(figsize=(10,8))
r, x_r = clust.get_crp_to_thermal_energy_profile(Emin=None, Emax=None)
plt.plot(r, x_r, label='Cosmic ray to thermal energy')
plt.vlines(clust.X_crp_E['R_norm'].to_value('kpc'), clust.X_crp_E['X']*0.9, clust.X_crp_E['X']*1.1, label='Imposed normalization')
plt.hlines(clust.X_crp_E['X'], clust.X_crp_E['R_norm'].to_value('kpc')*0.9, clust.X_crp_E['R_norm'].to_value('kpc')*1.1)
plt.xscale('log')
plt.xlabel('radius (kpc)')
plt.ylabel('$U_{CRp}/U_{th} (<r)$')
plt.legend()

In [ ]:
r1, p1 = clust.get_gamma_profile(radius=np.logspace(0,4,100)*u.kpc, 
                                   Emin=500*u.GeV, Emax=1*u.PeV, Energy_density=False)
r2, p2 = clust.get_gamma_profile(radius=np.logspace(0,4,100)*u.kpc, 
                                   Emin=1000*u.GeV, Emax=1*u.PeV, Energy_density=False)
plt.figure(0,figsize=(10,8))
plt.loglog((r1/clust.D_ang*u.rad).to('deg'),p1.to('cm-2 s-1 sr-1'), '-', label='500 GeV - 1 PeV')
plt.loglog((r2/clust.D_ang*u.rad).to('deg'),p2.to('cm-2 s-1 sr-1'), '--', label='1 TeV - 1 PeV')
plt.xlabel('Angular size (deg)')
plt.ylabel('Surface brightness ($ph/cm^2/s/sr$)')
plt.legend()

In [ ]:
energy = np.logspace(-2,5,100)*u.GeV
e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='spherical')
e3, s3 = clust.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='cylindrical')

# разом з СТА чутливістю
plt.figure(0,figsize=(10,8))
bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.37e-11, 8.35e-12, 2.02e-12, 9.41e-13, 5.37e-13, 3.58e-13, 2.43e-13, 1.76e-13, 1.35e-13, 1.24e-13, 1.1e-13,
        1.21e-13, 1.44e-13, 2.06e-13, 3.2e-13, 5.1e-13, 7.97e-13, 1.2e-12, 1.93e-12, 3.92e-12] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA North sensivity 50h')

#LHAASO sensitivity
bins_lo = [0.3, 0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300] # E_min (TeV)
bins_hi = [0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300, 600] # E_max (TeV)
data = [3e-9, 8e-10, 4e-10, 1e-10, 8e-11, 7e-11, 8e-11, 9e-11, 6e-11, 4e-11, 1.5e-11, 2e-11, 5e-11, 7e-11 ] # S (GeV/ cm^2 / s)
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data, color='r', ls='-.', lw='2', alpha=0.5, label = 'LHAASO sensivity')

plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), '-', label='Total volume A2151CB')
#plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='<R500, spherical integration')
#plt.loglog(e3, (e3**2*s3).to('GeV cm-2 s-1'), '-', label='<R500, cylindrical integration')
plt.gca().set_ylim(bottom= 1e-13)
plt.gca().set_ylim(top= 1e-7)
plt.gca().set_xlim(left= 1e-1)
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()

In [ ]:
# Defines the grid
clust.map_reso = 0.01*u.deg
clust.map_fov = 5.0*u.deg

# The center of the map can be offset wrt the cluster
clust.map_coord = SkyCoord("241.14d  17.72d", frame="icrs", unit="deg")

# Get the corresponding header
header = clust.get_map_header()

In [ ]:
template = clust.get_gamma_map(Emin=None, Emax=None, 
                               Energy_density=False, 
                               Normalize=True) # The map is normalized to the total flux, to be in unit of sr-1

In [ ]:
rad500 = clust.theta500.to_value('deg')
radtru = clust.theta_truncation.to_value('deg')

fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, projection=WCS(header))
plt.imshow(template, vmin=0, origin='lower', cmap='magma', norm=SymLogNorm(1))

# Show R500
circle500 = matplotlib.patches.Ellipse((clust.coord.ra.value, clust.coord.dec.value),
                                        2*rad500/np.cos(clust.coord.dec.value*np.pi/180), 2*rad500,
                                        linewidth=2, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
# Show R_truncation
circletru = matplotlib.patches.Ellipse((clust.coord.ra.value, clust.coord.dec.value),
                                        2*radtru/np.cos(clust.coord.dec.value*np.pi/180), 2*radtru,
                                        linewidth=1, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
ax.add_patch(circle500)
ax.add_patch(circletru)
plt.colorbar()
plt.title(clust.name+', gamma template ($sr^{-1}$)')

In [ ]:
# Save the parameters
clust.save_param()

# Save relevant output products
clust.save_spectra()
clust.save_profile()
clust.save_map()

# Can also produce plots
clust.plot(prod_list=['all'])

In [ ]:
clust_two = model.Cluster(name='A2151CF', 
                      redshift=0.0368, M500=3.01e13*u.Msun, 
                      cosmology=cosmo, silent=False, 
                      output_dir='/Users/vdk/Desktop/A2151CF_presentation')

In [ ]:
clust_two.coord = SkyCoord("241.28d  17.72d", frame="icrs", unit="deg")
clust_two.density_gas_model = {'name':'beta', 'n_0':0.0049*u.cm**-3, 'beta':0.28, 'r_c':10.43*u.kpc}
clust_two.pressure_gas_model = {'name':'GNFW', 'P_0':0.0077*u.keV/u.cm**3, 'c500':1.81, 'a':1.33, 'b':4.13, 'c':0.31}
clust_two.set_density_crp_isodens_scal_param()
clust_two.set_density_cre1_isodens_scal_param()
clust_two.spectrum_crp_model = {'name':'ExponentialCutoffPowerLaw', 'Index':1.5, 'CutoffEnergy':30*u.TeV}
clust_two.spectrum_cre1_model = {'name':'PowerLaw', 'Index':2.3}
clust_two.X_crp_E = {'X':0.1, 'R_norm':clust_two.R500} 
clust_two.X_cre1_E = {'X':0.01, 'R_norm':clust_two.R500} 
clust_two.set_magfield_isodens_scal_param(5*u.uG, scal=2/3)
clust_two.Epmax = 10e5*u.TeV
clust_two.Eemax = 10e5*u.TeV
clust_two.abundance = 0.13



In [ ]:
energy = np.logspace(-1,4,100)*u.GeV
e1, s1 = clust_two.get_gamma_spectrum(energy,Rmax=clust_two.R_truncation, type_integral='spherical')
e2, s2 = clust_two.get_gamma_spectrum(energy,Rmax=clust_two.R500, type_integral='spherical')
e3, s3 = clust_two.get_gamma_spectrum(energy,Rmax=clust_two.R500, type_integral='cylindrical')

# разом з СТА чутливістю
plt.figure(0,figsize=(10,8))
bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.37e-11, 8.35e-12, 2.02e-12, 9.41e-13, 5.37e-13, 3.58e-13, 2.43e-13, 1.76e-13, 1.35e-13, 1.24e-13, 1.1e-13,
        1.21e-13, 1.44e-13, 2.06e-13, 3.2e-13, 5.1e-13, 7.97e-13, 1.2e-12, 1.93e-12, 3.92e-12] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA North sensivity 50h')

#LHAASO sensitivity
bins_lo = [0.3, 0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300] # E_min (TeV)
bins_hi = [0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300, 600] # E_max (TeV)
data = [3e-9, 8e-10, 4e-10, 1e-10, 8e-11, 7e-11, 8e-11, 9e-11, 6e-11, 4e-11, 1.5e-11, 2e-11, 5e-11, 7e-11 ] # S (GeV/ cm^2 / s)
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data, color='r', ls='-.', lw='2', alpha=0.5, label = 'LHAASO sensivity')

plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), '-', label='Total volume')
plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='<R500, spherical integration')
plt.loglog(e3, (e3**2*s3).to('GeV cm-2 s-1'), '-', label='<R500, cylindrical integration')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()

In [ ]:
# Defines the grid
clust_two.map_reso = 0.01*u.deg
clust_two.map_fov = 5.0*u.deg

# The center of the map can be offset wrt the cluster
clust_two.map_coord = SkyCoord("241.28d  17.72d", frame="icrs", unit="deg")

# Get the corresponding header
header = clust_two.get_map_header()

In [ ]:
template = clust_two.get_gamma_map(Emin=None, Emax=None, 
                               Energy_density=False, 
                               Normalize=True) # The map is normalized to the total flux, to be in unit of sr-1

In [ ]:
rad500 = clust_two.theta500.to_value('deg')
radtru = clust_two.theta_truncation.to_value('deg')

fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, projection=WCS(header))
plt.imshow(template, vmin=0, origin='lower', cmap='magma', norm=SymLogNorm(1))

# Show R500
circle500 = matplotlib.patches.Ellipse((clust_two.coord.ra.value, clust_two.coord.dec.value),
                                        2*rad500/np.cos(clust_two.coord.dec.value*np.pi/180), 2*rad500,
                                        linewidth=2, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
# Show R_truncation
circletru = matplotlib.patches.Ellipse((clust_two.coord.ra.value, clust_two.coord.dec.value),
                                        2*radtru/np.cos(clust_two.coord.dec.value*np.pi/180), 2*radtru,
                                        linewidth=1, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
ax.add_patch(circle500)
ax.add_patch(circletru)
plt.colorbar()
plt.title(clust_two.name+', gamma template ($sr^{-1}$)')

In [ ]:
# Save the parameters
clust_two.save_param()

# Save relevant output products
clust_two.save_spectra()
clust_two.save_profile()
clust_two.save_map()

# Can also produce plots
clust_two.plot(prod_list=['all'])

In [ ]:
bins_lo = [0.3, 0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300] # E_min (TeV)
bins_hi = [0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300, 600] # E_max (TeV)
data = [3e-9, 8e-10, 4e-10, 1e-10, 8e-11, 7e-11, 8e-11, 9e-11, 6e-11, 4e-11, 1.5e-11, 2e-11, 5e-11, 7e-11 ] # S (GeV/ cm^2 / s)
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data, color='r', ls='-.', lw='2', alpha=0.5)
plt.show()



In [ ]:
x = [0.3, 0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300, 600]
y = [3e-9, 8e-10, 4e-10, 1e-10, 8e-11, 7e-11, 8e-11, 9e-11, 6e-11, 4e-11, 1.5e-11, 2e-11, 5e-11, 7e-11, 1e-10]
plt.plot(x,y)
plt.show()

In [ ]:
bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.37e-11, 8.35e-12, 2.02e-12, 9.41e-13, 5.37e-13, 3.58e-13, 2.43e-13, 1.76e-13, 1.35e-13, 1.24e-13, 1.1e-13,
        1.21e-13, 1.44e-13, 2.06e-13, 3.2e-13, 5.1e-13, 7.97e-13, 1.2e-12, 1.93e-12, 3.92e-12] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
len(bins_hi)

In [ ]:
energy = np.logspace(-1,5,100)*u.GeV
e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust_two.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')

# разом з СТА чутливістю
plt.figure(0,figsize=(10,8))
bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.37e-11, 8.35e-12, 2.02e-12, 9.41e-13, 5.37e-13, 3.58e-13, 2.43e-13, 1.76e-13, 1.35e-13, 1.24e-13, 1.1e-13,
        1.21e-13, 1.44e-13, 2.06e-13, 3.2e-13, 5.1e-13, 7.97e-13, 1.2e-12, 1.93e-12, 3.92e-12] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA North sensivity 50h')

#LHAASO sensitivity
bins_lo = [0.3, 0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300] # E_min (TeV)
bins_hi = [0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300, 600] # E_max (TeV)
data = [3e-9, 8e-10, 4e-10, 1e-10, 8e-11, 7e-11, 8e-11, 9e-11, 6e-11, 4e-11, 1.5e-11, 2e-11, 5e-11, 7e-11 ] # S (GeV/ cm^2 / s)
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data, color='r', ls='-.', lw='2', alpha=0.5, label = 'LHAASO sensivity')

sum = s1 + s2
plt.gca().set_ylim(bottom= 1e-12)
plt.gca().set_xlim(left= 1e-1)

plt.loglog(e1, (e1**2*sum).to('GeV cm-2 s-1'), '-', label='Total volume of two subclumps')
#plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='Total volume')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()

In [ ]:
print(s1+s2)

In [ ]:
print(s2)

In [ ]:
energy = np.logspace(-1,5,100)*u.GeV
e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust_two.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')

# разом з СТА чутливістю
plt.figure(0,figsize=(10,8))
bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.37e-11, 8.35e-12, 2.02e-12, 9.41e-13, 5.37e-13, 3.58e-13, 2.43e-13, 1.76e-13, 1.35e-13, 1.24e-13, 1.1e-13,
        1.21e-13, 1.44e-13, 2.06e-13, 3.2e-13, 5.1e-13, 7.97e-13, 1.2e-12, 1.93e-12, 3.92e-12] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA North sensivity 50h')

#LHAASO sensitivity
bins_lo = [0.3, 0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300] # E_min (TeV)
bins_hi = [0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300, 600] # E_max (TeV)
data = [3e-9, 8e-10, 4e-10, 1e-10, 8e-11, 7e-11, 8e-11, 9e-11, 6e-11, 4e-11, 1.5e-11, 2e-11, 5e-11, 7e-11 ] # S (GeV/ cm^2 / s)
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data, color='r', ls='-.', lw='2', alpha=0.5, label = 'LHAASO sensivity')

sum = s1 + s2
plt.gca().set_ylim(bottom= 1e-12)
plt.gca().set_xlim(left= 1e-1)
plt.loglog(e1, (e1**2*sum).to('GeV cm-2 s-1'), '-', label='Total volume of two subclumps')
#plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='Total volume')

plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()


In [ ]:
print(energy)

In [ ]:
r1, p1 = clust.get_pressure_gas_profile()

plt.figure(0,figsize=(10,8))
plt.loglog(r1,p1, '.', label = 'Pressure profile')
plt.xlabel('Radius, kpc')
plt.ylabel('Pressure, keV/cm^3')
plt.legend()

In [ ]:
r = [15, 45, 75, 105, 135, 165]
P = [14e-3, 9.26e-3, 5.6e-3, 6.14e-3, 3.75e-3, 2.6e-3]
r1, p1 = clust.get_pressure_gas_profile()


plt.figure(0,figsize=(10,8))
plt.loglog(r1,p1, '-', label = 'Pressure profile MINOT')
plt.loglog(r,P, '-', label = 'Pressure profile Tiwari')
plt.xlabel('Radius, kpc')
plt.ylabel('Pressure, keV/cm^3')
plt.gca().set_ylim(bottom= 1e-3)
plt.gca().set_xlim(left= 10)
plt.legend()


In [ ]:
clust.plot().__dir__()

In [ ]:
clust.print_param()

In [ ]:
cosmo

In [ ]:
def plot_Fermi_sens():

    #Fermi
    bins_lo = [5e-5, 7e-5, 2e-4, 4e-4, 6e-4, 1.5e-3, 2e-3, 4e-3, 8e-3,5e-2] #in TeV
    bins_hi = [7e-5, 2e-4, 4e-4, 6e-4, 1.5e-3, 2e-3, 4e-3, 8e-3,5e-2,0.1] #in TeV
    data = [6e-12, 3e-12, 1e-12, 8e-13, 6.5e-13, 3.75e-13, 3e-13, 3.5e-13, 5e-13, 2e-12] # S (erg / cm^2 / s)
    data_GeV = [i * 624.15064799632 for i in data]
    mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]


    plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5)
    plt.show()

plot_Fermi_sens()